# JS 如何通过事件循环机制（EventLoop）实现异步执行

JS 是一种单线程的同步语言，这意味着在任何时候，JS 引擎中的调用栈只能执行一个任务。

调用栈是一个栈结构，存在于 JS 引擎，用于管理 JS 执行上下文的执行顺序。

前面文章介绍过：

> 当 V8 执行全局/模块/函数代码的时候，在创建阶段会创建相应的执行上下文。
> 执行上下文的状态组件「变量环境组件」和「词法环境组件」记录着当前词法环境的变量和函数。

当一个新的执行上下文被创建时，它会被压入调用栈，JS 引擎总是执行栈顶的上下文。当当前上下文的代码执行完毕后，它会从调用栈中弹出，控制权返回到上一个上下文。如果一个函数创建了闭包，尽管这个函数执行完毕后其执行上下文会从调用栈中弹出，但其词法环境依然会被闭包所引用，因此不会被垃圾回收机制立即回收。

用一个实际例子说明调用栈如何管理执行上下文：


In [3]:
function first() {
  console.log(1);
  second();
}

function second() {
  console.log(2);
  third();
}

function third() {
  console.log(3);
}

first();


1
2
3


在讲事件循环机制之前先插入几个概念。1


### 运行时（runtime）

JS 运行时全称 JS 运行时环境，是 JS 代码的执行环境，由 JS 引擎、事件循环机制、回调队列和宿主 APIs 组成。

JS 引擎通过调用栈和堆执行代码和管理内存。

- 调用栈：
- 内存堆：

宿主是浏览器、Node


执行上下文被创建之后会被压入调用栈，调用栈是跟踪代码执行流程的数据结构，当代码开始执行的时候会首先创建全局执行上下文并入栈，
执行到函数的时候会创建函数执行上下文并入栈，调用栈只执行栈顶的执行上下文代码，栈顶函数执行完就出栈，继续执行调用栈的栈顶函数。

在执行上下文里，基本类型的变量值存在环境记录里，引用类型和函数的值存在堆内存里，环境记录里存的是堆的内存地址。
